In [1]:
import sys
sys.path.append("..")

from speech_encoder_v2_updated import SpeechEncoderV2
from params import *
from pathlib import Path
import torch
import utils
import visualisations
from data_scripts import *

In [2]:
def sync(device: torch.device):
    # For correct profiling (cuda operations are async)
    if device.type == "cuda":
        torch.cuda.synchronize(device)

In [3]:
params = {
    "run_id": "speech_encoder_transformer_updated",  # A unique identifier for this training run
    "clean_data_root": "D:/CODING/SpeechEncoder/data/his_processed_audio",  # Path to LibriSpeech dataset
    "models_dir": "models",  # Directory to save model checkpoints
    "umap_every": 500,  # Update UMAP visualization every 500 steps
    "save_every": 500,  # Save model checkpoint every 500 steps
    "backup_every": 5000,  # Create a backup copy of the model every 5000 steps
    "vis_every": 100,  # Update visualization metrics every 100 steps
    "force_restart": False,  # Whether to restart training from scratch
    "visdom_server": "http://localhost",  # Visdom server address for visualization
    "no_visdom": False,  # Whether to disable Visdom visualization
    "models_dir": Path("../models"),  # Directory to save model checkpoints
}

In [4]:
dataset, loader = load_data(params['clean_data_root'], 40, 10, 0)

In [5]:
for batch in loader:
    print(batch.data.shape)
    break  # Check the shape of the first batch only

(400, 160, 40)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_device = torch.device("cpu")

# Create the model and the optimizer
model = SpeechEncoderV2(device, device)
model.to(device)

#Uncomment the bellow lines to load the model from a checkpoint

checkpoints = torch.load("../models\speech_encoder_transformer_updated\encoder_031000_loss_0.2147.pt")
model.load_state_dict(checkpoints['model_state'])

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate_init)
init_step = 31267

# Configure file path for the model
model_dir = params['models_dir'] / params['run_id']
model_dir.mkdir(exist_ok=True, parents=True)
state_fpath = model_dir / "encoder.pt"

c:\Users\anike\anaconda3\envs\gpu_environment\lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [8]:
import sys
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

log_dir = params['models_dir'] / "logs"

# Initialize TensorBoard writer (ensure log_dir is defined)
writer = SummaryWriter(log_dir=log_dir)

# Initialize the progress bar
total_steps = len(loader)  # Assuming `loader` has a defined length
progress_bar = tqdm(enumerate(loader, init_step), total=total_steps, desc="Training", unit="step")

model.train()

for step, speaker_batch in progress_bar:
    # Forward pass
    inputs = torch.from_numpy(speaker_batch.data).to(device)
    sync(device)
    embeds = model(inputs)
    sync(device)
    embeds_loss = embeds.view((speakers_per_batch, utterances_per_speaker, -1)).to(loss_device)
    loss, eer = model.loss(embeds_loss)
    sync(loss_device)

    # Backward pass
    model.zero_grad()
    loss.backward()
    
    model.do_gradient_ops()
    optimizer.step()
    
    # Log scalars to TensorBoard
    writer.add_scalar("Loss", loss.item(), step)
    writer.add_scalar("EER", eer, step)
    
    # Update the progress bar with the current loss and EER
    progress_bar.set_postfix({"loss": loss.item(), "eer": eer})

    # Save the model every 'save_every' steps with a unique filename that includes the step and loss
    if params['save_every'] != 0 and step % params['save_every'] == 0:
        filename = model_dir / f"encoder_{step:06d}_loss_{loss.item():.4f}.pt"
        print("Saving the model (step %d) to %s" % (step, filename))
        torch.save({
            "step": step + 1,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
        }, filename)

    # Make a backup every 'backup_every' steps
    if params['backup_every'] != 0 and step % params['backup_every'] == 0:
        print("Making a backup (step %d)" % step)
        backup_fpath = model_dir / f"encoder_{step:06d}.bak"
        torch.save({
            "step": step + 1,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
        }, backup_fpath)
    
    # Update the progress bar with loss and EER information.
    progress_bar.set_postfix(loss=loss.item(), eer=eer)
    
# Optionally, close the writer after training
writer.close()


Training:   0%|          | 234/250000000 [07:21<53407:41:50,  1.30step/s, eer=0.03, loss=0.292]  

Saving the model (step 31500) to ..\models\speech_encoder_transformer_updated\encoder_031500_loss_0.2925.pt


Training:   0%|          | 734/250000000 [13:22<48147:38:33,  1.44step/s, eer=0.0275, loss=0.208]

Saving the model (step 32000) to ..\models\speech_encoder_transformer_updated\encoder_032000_loss_0.2082.pt


Training:   0%|          | 1234/250000000 [18:51<45155:39:19,  1.54step/s, eer=0.03, loss=0.29]   

...

Saving the model (step 73000) to ..\models\speech_encoder_transformer_updated\encoder_073000_loss_0.0852.pt


Training:   0%|          | 42234/250000000 [8:39:33<56502:06:58,  1.23step/s, eer=0.00904, loss=0.0724]

Saving the model (step 73500) to ..\models\speech_encoder_transformer_updated\encoder_073500_loss_0.0724.pt


Training:   0%|          | 42513/250000000 [8:43:19<51282:33:15,  1.35step/s, eer=0.00981, loss=0.0695]


KeyboardInterrupt: 